In [1]:
print("hello world")

hello world


In [5]:
%pip install -r requirements.txt

Obtaining file:///D:/Medical%20Chatbot/Medical-Chatbot-with-LLMs-LangChain-Pinecone-Flask-AWS (from -r requirements.txt (line 16))
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_pinecone-0.2.8-py3-none-any.whl.metadata (5.3 kB)
  Using cached langchain_openai-0.3.24-py3-none-any.whl.metadata (2.3 kB)
  Using cached langchain_community-0.3.26-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_core-0.3.80-py3-none-any.whl.metadata (3.2 kB)
  Using 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-classic 1.0.0 requires langchain-text-splitters<2.0.0,>=1.0.0, but you have langchain-text-splitters 0.3.11 which is incompatible.
langchain-google-vertexai 3.2.0 requires langchain-core<2.0.0,>=1.1.1, but you have langchain-core 0.3.80 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.


In [6]:
%cd "D:/Medical Chatbot/Medical-Chatbot-with-LLMs-LangChain-Pinecone-Flask-AWS"


D:\Medical Chatbot\Medical-Chatbot-with-LLMs-LangChain-Pinecone-Flask-AWS


In [7]:
%pwd

'D:\\Medical Chatbot\\Medical-Chatbot-with-LLMs-LangChain-Pinecone-Flask-AWS'

In [8]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PDFPlumberLoader

c:\ProgramData\anaconda3\envs\medicalbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
#Extract text from PDF
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="**/*.pdf",
        loader_cls=PDFPlumberLoader
    )
    documents=loader.load()
    return documents

In [6]:
extracted_data=load_pdf_files("data")

In [35]:
from typing import List
from langchain.schema import Document

c:\ProgramData\anaconda3\envs\medicalbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [155]:
len(extracted_data)

637

In [37]:
def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs: List[Document] = []
    for idx, doc in enumerate(docs):
        src = doc.metadata.get("source")
        page = doc.metadata.get("page")

        minimal_docs.append(
            Document(
                page_content=doc.page_content.strip(),
                metadata={
                    "source": src,
                    "page": page,
                    "chunk_id": idx
                }
            )
        )
    return minimal_docs


In [38]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [40]:
# Tăng overlap để giữ mạch văn
def text_splitter(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100, # Tăng từ 20 lên 100
        separators=["\n\n", "\n", ". ", " ", ""] # Ưu tiên cắt hết câu
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [42]:
texts_chunk=text_splitter(minimal_docs)

In [ ]:
#from langchain.embeddings import HuggingFaceEmbeddings
#import torch
#def download_embeddings():
    # Use a widely available sentence-transformers model on Hugging Face
#    embeddings = HuggingFaceEmbeddings(
#        model_name="sentence-transformers/all-MiniLM-L6-v2",
#        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
#    )
#    return embeddings

In [43]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

def download_embeddings():
    model_name = "intfloat/multilingual-e5-base"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"},
        encode_kwargs={"normalize_embeddings": True}
    )
    return embeddings


In [44]:
embeddings=download_embeddings()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18888\2018390847.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [45]:
vector=embeddings.embed_query("HUST IS GOOD.")
vector

[0.009380968287587166,
 0.0008409664733335376,
 -0.03295055404305458,
 0.05460749566555023,
 0.026847733184695244,
 -0.013549319468438625,
 -0.016637714579701424,
 -0.0031920969486236572,
 0.04838130995631218,
 0.03673918545246124,
 -0.03980075195431709,
 -0.004988291300833225,
 0.11221061646938324,
 0.016348998993635178,
 -0.02261929214000702,
 -0.057567205280065536,
 0.008441057056188583,
 -0.0186668261885643,
 0.01591019332408905,
 -0.02262086421251297,
 0.0003326952864881605,
 -0.024052688851952553,
 0.030050721019506454,
 0.0042272768914699554,
 0.048479825258255005,
 -0.02305305376648903,
 0.00421969685703516,
 0.008009013719856739,
 -0.05056139826774597,
 0.004855398554354906,
 0.03167636692523956,
 -0.0081698102876544,
 0.03779512271285057,
 0.04353725165128708,
 0.05651077255606651,
 0.02951444312930107,
 -0.002666948828846216,
 -0.061967819929122925,
 0.031758785247802734,
 0.0226795244961977,
 0.015231502242386341,
 0.029597623273730278,
 0.02337043173611164,
 -0.03965871408

In [46]:
dimension=len(vector)
dimension

768

In [47]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [189]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")
COHERE_API_KEY=os.getenv("COHERE_API_KEY")
OPEN_AI_API_KEY=os.getenv("OPEN_AI_API_KEY")

os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GEMINI_API_KEY"]=GEMINI_API_KEY
os.environ["COHERE_API_KEY"]=COHERE_API_KEY



In [49]:
from pinecone import Pinecone

pinecone_api_key=PINECONE_API_KEY

pc=Pinecone(api_key=pinecone_api_key)
pc

In [50]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

index_name = "medical-chatbot"



existing_indexes = [index["name"] for index in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

index = pc.Index(index_name)


In [21]:
from langchain_pinecone import PineconeVectorStore
from tqdm import tqdm

# Khởi tạo vector store 1 lần duy nhất
vectorstore = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings,
    text_key="page_content"
)

batch_size = 100

print(f"Uploading {len(texts_chunk)} chunks to Pinecone...")

for i in tqdm(range(0, len(texts_chunk), batch_size)):
    batch = texts_chunk[i:i + batch_size]


    vectorstore.add_documents(
        documents=batch,
        ids=None  
    )

print("Successfully uploaded all documents to Pinecone.")


Uploading 5898 chunks to Pinecone...


100%|██████████| 59/59 [18:30<00:00, 18.82s/it]

Successfully uploaded all documents to Pinecone.


In [51]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
    text_key="page_content"
)


In [52]:
dswith=Document(
    page_content="Sample content",
    metadata={"source": "sample.pdf", "page": 1}
)

In [53]:
docsearch.add_documents([dswith])

['1fd2e96d-cf77-4e30-80ed-9ec569e4b2e3']

In [54]:
dense_retriever = docsearch.as_retriever(
    search_kwargs={"k": 10}
)


In [27]:
%pip install rank_bm25

In [55]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(
    texts_chunk,
    k=20
)


In [56]:
from langchain.retrievers import EnsembleRetriever

retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, dense_retriever],
    weights=[0.4, 0.6]
)


In [45]:
retriever_docs=retriever.invoke("What is HIV?")
retriever_docs

[Document(id='762affac-cb9a-438a-af93-b366e9d219f0', metadata={'chunk_id': 91.0, 'page': 91.0, 'source': 'data\\Medical_book.pdf'}, page_content='protein or particle called an antigen. Human immunodeficiency virus (HIV)—A transmis-\nAntigen—Any substance that stimulates the body to sible retrovirus that causes AIDS in humans. Two\nproduce antibody. forms of HIV are now recognized: HIV-1, which caus-\nes most cases of AIDS in Europe, North and South\nAutoimmunity—A condition in which the body’s\nAmerica, and most parts of Africa; and HIV-2, which\nimmune system produces antibodies in response to'),
 Document(id='1411bdd9-557a-4803-adba-710b076aa11c', metadata={'chunk_id': 95.0, 'page': 95.0, 'source': 'data\\Medical_book.pdf'}, page_content='the immune system. HIV is a retrovirus that enters the\nscreen patients and confirm diagnoses.\nblood stream of a new host in the following ways:\n•by sexual contact ENZYME-LINKED IMMUNOSORBENT ASSAY (ELISA).\nThis type of blood test is used to scre

In [35]:
%pip install cohere

  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)

  Attempting uninstall: httpx-sse

    Found existing installation: httpx-sse 0.4.3

    Uninstalling httpx-sse-0.4.3:

      Successfully uninstalled httpx-sse-0.4.3

   -------------------- ------------------- 2/4 [fastavro]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ------------------------------ --------- 3/4 [cohere]
   ---------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.13 requires langchain-core<0.4.0,>=0.3.27, but you have langchain-core 0.3.13 which is incompatible.


In [58]:
import cohere
from langchain.schema import Document

cohere_api_key=os.getenv("COHERE_API_KEY")
cohere_client = cohere.Client(cohere_api_key)

In [234]:
# --- NHẬP CÂU HỎI CỦA BẠN ---
user_query = "Ung thư máu là gì?"
print(f"🏥 Câu hỏi: {user_query}")

🏥 Câu hỏi: Ung thư máu là gì?


In [235]:
# 2. TÌM KIẾM (Hybrid Search: BM25 + Vector)
print("   ↳ 🔍 Đang tìm tài liệu (Hybrid Search)...")
# 'retriever' là biến EnsembleRetriever bạn đã tạo ở Cell 56
initial_docs = retriever.invoke(user_query)

   ↳ 🔍 Đang tìm tài liệu (Hybrid Search)...


In [236]:
# 3. RERANK (Sắp xếp lại bằng Cohere)
print(f"   ↳ ⚖️ Đang chấm điểm {len(initial_docs)} tài liệu...")
passages = [doc.page_content for doc in initial_docs]
rerank_results = cohere_client.rerank(
    model="rerank-multilingual-v3.0",
    query=user_query,
    documents= passages,
    top_n=10
)
final_docs = [initial_docs[r.index] for r in rerank_results.results]


   ↳ ⚖️ Đang chấm điểm 25 tài liệu...


In [243]:

if 'final_docs' in globals() and final_docs:
    context_text = "\n\n---\n\n".join([
        f"[Nguồn: {doc.metadata.get('source', 'Unknown')} - Tr {doc.metadata.get('page', '?')}]\n{doc.page_content}" 
        for doc in final_docs
    ])
else:
    context_text = "Không có tài liệu nào được tìm thấy."

In [245]:
# 3. Tạo Prompt (Kịch bản tối ưu hóa độ dài và chi tiết)
system_prompt = """
Bạn là Trợ lý Y khoa AI chuyên sâu và tận tâm.
Nhiệm vụ của bạn là tổng hợp thông tin từ tài liệu (Context) để trả lời người dùng một cách CHI TIẾT và DỄ HIỂU.

NGUYÊN TẮC TRẢ LỜI:
1.  **Phân tích sâu:** Đừng chỉ liệt kê. Hãy giải thích rõ cơ chế, nguyên nhân, và mối liên hệ giữa các triệu chứng/bệnh lý nếu tài liệu có đề cập.
2.  **Cấu trúc rõ ràng:**
    * Sử dụng gạch đầu dòng (bullet points) cho các danh sách.
    * Chia câu trả lời thành các mục nhỏ (Ví dụ: Định nghĩa, Triệu chứng, Nguyên nhân, Cách xử lý...).
3.  **Ngôn ngữ:**
    * Trả lời hoàn toàn bằng Tiếng Việt.
    * Giữ nguyên thuật ngữ tiếng Anh quan trọng (kèm giải thích tiếng Việt trong ngoặc).
    * Văn phong chuyên nghiệp nhưng ân cần, đồng cảm.
4.  **Trích dẫn nguồn:** Cuối câu trả lời, hãy ghi chú thông tin được lấy từ tài liệu nào (Dựa trên metadata source/page trong Context).
5.  **Giới hạn an toàn:**
    * Chỉ dùng thông tin trong Context.
    * Nếu Context thiếu thông tin, hãy nói rõ: "Tài liệu hiện tại chưa cung cấp đủ thông tin về vấn đề này".
    * Luôn kèm câu miễn trừ trách nhiệm: "Thông tin chỉ mang tính tham khảo, vui lòng tham khảo ý kiến bác sĩ chuyên khoa."

Hãy đóng vai một bác sĩ đang giải thích cặn kẽ cho bệnh nhân hiểu rõ vấn đề.
"""

user_message = f"""
CÂU HỎI CỦA NGƯỜI DÙNG:
"{user_query}"

---
DỮ LIỆU THAM KHẢO (CONTEXT):
{context_text}
---

Hãy trả lời câu hỏi trên thật chi tiết dựa vào Dữ liệu tham khảo:
"""

In [246]:
# 4. Gọi GPT-4o-mini để trả lời
from openai import OpenAI
from langchain_openai import ChatOpenAI
import getpass

user_key = getpass.getpass("👉 Paste OpenAI API Key: ")

client = OpenAI(api_key=user_key)

# 4. Gọi OpenAI (Nhớ set max_tokens cao để không bị cắt)
print("🤖 Bác sĩ đang tổng hợp thông tin chi tiết...")

try:
    response = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ],
        temperature=0.5, # Tăng nhẹ để văn phong mượt mà hơn (nhưng vẫn < 0.7 để an toàn)
        max_tokens=1500  # Cho phép trả lời dài (khoảng 1 trang A4)
    )
    
    # 5. In kết quả
    print("\n" + "="*40)
    print("💡 KẾT QUẢ TƯ VẤN CHI TIẾT:")
    print("="*40)
    print(response.choices[0].message.content)

except Exception as e:
    print(f"❌ Lỗi khi gọi OpenAI: {e}")

🤖 Bác sĩ đang tổng hợp thông tin chi tiết...

💡 KẾT QUẢ TƯ VẤN CHI TIẾT:
### Định nghĩa Ung thư máu

Ung thư máu, hay còn gọi là **Leukemia**, là một loại ung thư ảnh hưởng đến các tế bào máu, chủ yếu là tế bào bạch cầu (white blood cells). Đặc điểm nổi bật của bệnh này là sự gia tăng bất thường về số lượng tế bào bạch cầu trong cơ thể. Điều này có thể dẫn đến việc các tế bào này không hoạt động hiệu quả, gây ra nhiều vấn đề sức khỏe.

### Triệu chứng của Ung thư máu

Mặc dù tài liệu không cung cấp chi tiết về triệu chứng cụ thể của ung thư máu, nhưng thường thì các triệu chứng có thể bao gồm:

- **Mệt mỏi**: Do thiếu hụt tế bào máu khỏe mạnh.
- **Sốt**: Do cơ thể bị nhiễm trùng do sự thiếu hụt tế bào bạch cầu.
- **Chảy máu hoặc bầm tím dễ dàng**: Do thiếu hụt tiểu cầu (platelets) cần thiết cho quá trình đông máu.
- **Sụt cân không rõ nguyên nhân**: Do cơ thể không thể hấp thụ dinh dưỡng hiệu quả.

### Nguyên nhân gây ra Ung thư máu

Nguyên nhân chính xác của ung thư máu vẫn chưa được 